In [1]:
import numpy as np
import pandas as pd
import re
import warnings
warnings.filterwarnings("ignore")

# Permet d'afficher toutes les colonnes
pd.set_option('display.max_columns', None)

In [2]:
water_important_cols = [
    'row_index',
    'meteo_rain_height',
    'meteo_evapotranspiration_grid',
    'meteo_evapotranspiration_Monteith',
    'meteo_sunshine_%',
    'meteo_sunshine_duration',
    'meteo_temperature_avg',
    'meteo_temperature_min',
    'meteo_temperature_max',
    'meteo_amplitude_tn_tx',
    'meteo_humidity_avg',
    'meteo_humidity_min',
    'meteo_pressure_avg',
    'meteo_cloudiness',
    'meteo_frost_duration',
    'meteo_wetting_duration',
    'meteo_humidity_duration_below_40%',
    'meteo_humidity_duration_above_80%',
    'meteo_radiation_direct',
    'meteo_radiation',
    'meteo_radiation_IR',
    'meteo_if_snow',
    'meteo_snow_height',
    'meteo_snow_thickness_6h',
    'meteo_snow_thickness_max'
]

water_important_cols_first_part = [
    'row_index',
    'meteo_rain_height',
    'meteo_evapotranspiration_grid',
    'meteo_evapotranspiration_Monteith',
    'meteo_sunshine_%',
    'meteo_sunshine_duration',
    'meteo_temperature_avg',
    'meteo_temperature_min',
    'meteo_temperature_max',
    'meteo_amplitude_tn_tx',
    'meteo_humidity_avg',
    'meteo_humidity_min',
    'meteo_pressure_avg'
]

In [3]:
data_meteo_part = pd.read_csv("X_test_Hi5.csv", usecols=water_important_cols_first_part)

In [4]:
data_meteo_part['meteo_amplitude_tn_tx'] = data_meteo_part['meteo_amplitude_tn_tx'].fillna(data_meteo_part['meteo_temperature_max'] - data_meteo_part['meteo_temperature_min'])
data_meteo_part['meteo_temperature_min'] = data_meteo_part['meteo_temperature_min'].fillna(data_meteo_part['meteo_temperature_max'] - data_meteo_part['meteo_amplitude_tn_tx'])
data_meteo_part['meteo_temperature_max'] = data_meteo_part['meteo_temperature_max'].fillna(data_meteo_part['meteo_temperature_min'] + data_meteo_part['meteo_amplitude_tn_tx'])

In [5]:
data_meteo_part.isna().sum()

row_index                                 0
meteo_rain_height                      7052
meteo_temperature_min                 19406
meteo_temperature_max                 19419
meteo_temperature_avg                 25528
meteo_amplitude_tn_tx                 19446
meteo_pressure_avg                   522882
meteo_humidity_min                   307547
meteo_humidity_avg                   307279
meteo_sunshine_duration              533143
meteo_sunshine_%                     533143
meteo_evapotranspiration_Monteith    508544
meteo_evapotranspiration_grid         27875
dtype: int64

In [6]:
df_cleaned = data_meteo_part.dropna(subset=['meteo_temperature_min','meteo_temperature_avg','meteo_temperature_max','meteo_rain_height','meteo_evapotranspiration_grid','meteo_humidity_min','meteo_humidity_avg'])
df_cleaned.drop(columns=['meteo_amplitude_tn_tx'], inplace = True)

In [7]:
df_cleaned.isna().sum()

row_index                                 0
meteo_rain_height                         0
meteo_temperature_min                     0
meteo_temperature_max                     0
meteo_temperature_avg                     0
meteo_pressure_avg                   209031
meteo_humidity_min                        0
meteo_humidity_avg                        0
meteo_sunshine_duration              219303
meteo_sunshine_%                     219303
meteo_evapotranspiration_Monteith    200633
meteo_evapotranspiration_grid             0
dtype: int64

In [8]:
df_cleaned.describe()

,row_index,meteo_rain_height,meteo_temperature_min,meteo_temperature_max,meteo_temperature_avg,meteo_pressure_avg,meteo_humidity_min,meteo_humidity_avg,meteo_sunshine_duration,meteo_sunshine_%,meteo_evapotranspiration_Monteith,meteo_evapotranspiration_grid
count,2.972790e+05,297279.000000,297279.000000,297279.000000,297279.000000,88248.000000,297279.000000,297279.000000,77976.000000,77976.000000,96646.000000,297279.000000
mean,2.968739e+06,1.771886,15.051677,27.464436,20.956004,1015.930028,41.936760,67.513965,519.088720,60.544770,4.882660,4.860522
std,4.881762e+05,5.259438,3.921022,5.015078,3.964739,4.352021,14.171098,13.436562,246.040058,28.017052,1.789659,1.805097
min,2.331795e+06,0.000000,-3.000000,4.400000,2.600000,991.000000,4.000000,24.000000,0.000000,0.000000,0.100000,0.200000
25%,2.492423e+06,0.000000,12.600000,24.000000,18.400000,1013.400000,32.000000,58.000000,320.000000,37.000000,3.600000,3.600000
50%,2.652420e+06,0.000000,15.200000,27.400000,20.900000,1016.000000,41.000000,69.000000,557.000000,65.000000,4.800000,4.700000
75%,3.451314e+06,0.600000,17.600000,30.900000,23.700000,1018.800000,52.000000,78.000000,725.000000,87.000000,6.000000,6.000000
max,3.610822e+06,281.700000,30.400000,43.300000,33.800000,1033.000000,100.000000,100.000000,919.000000,97.000000,9.000000,9.000000


In [9]:
# Définir des tranches pour chaque colonne
bins_avg = [0, 15, 20, 25, 35]  # Tranches pour la température moyenne
bins_min = [-5, 5, 10, 15, 20, 35]  # Tranches pour la température minimale
bins_max = [0, 20, 30, 40, 50]  # Tranches pour la température maximale
bins_humidity_avg = [0, 20, 40, 60, 80, 100]  # Tranches pour l'humidité moyenne
bins_humidity_min = [0, 20, 40, 60, 80, 100]  # Tranches pour l'humidité minimale

labels_avg = ['0-15', '15-20', '20-25', '25-35']
labels_min = ['-5-5', '5-10', '10-15', '15-20', '20-35']
labels_max = ['0-20', '20-30', '30-40', '40-50']
labels_humidity_avg = ['0-20', '20-40', '40-60', '60-80', '80-100']
labels_humidity_min = ['0-20', '20-40', '40-60', '60-80', '80-100']

# Ajouter des colonnes de tranches
df_cleaned['temp_range_avg'] = pd.cut(df_cleaned['meteo_temperature_avg'], bins=bins_avg, labels=labels_avg, right=False)
df_cleaned['temp_range_min'] = pd.cut(df_cleaned['meteo_temperature_min'], bins=bins_min, labels=labels_min, right=False)
df_cleaned['temp_range_max'] = pd.cut(df_cleaned['meteo_temperature_max'], bins=bins_max, labels=labels_max, right=False)
df_cleaned['humidity_avg_range'] = pd.cut(df_cleaned['meteo_humidity_avg'], bins=bins_humidity_avg, labels=labels_humidity_avg, right=False)
df_cleaned['humidity_min_range'] = pd.cut(df_cleaned['meteo_humidity_min'], bins=bins_humidity_min, labels=labels_humidity_min, right=False)

# Regrouper par les colonnes de tranches et calculer les moyennes
group_cols_sunshine = ['temp_range_avg', 'temp_range_min', 'temp_range_max']
group_cols_evapotranspiration = ['temp_range_avg', 'humidity_avg_range', 'humidity_min_range']

sunshine_avg = df_cleaned.groupby(group_cols_sunshine)['meteo_sunshine_%'].mean()
duration_avg = df_cleaned.groupby(group_cols_sunshine)['meteo_sunshine_duration'].mean()
pressure_avg = df_cleaned.groupby('temp_range_avg')['meteo_pressure_avg'].mean()
evapotranspiration_avg = df_cleaned.groupby(group_cols_evapotranspiration)['meteo_evapotranspiration_Monteith'].mean()

# Remplir les NaN pour meteo_sunshine_% et meteo_sunshine_duration
df_cleaned['meteo_sunshine_%'] = df_cleaned.apply(
    lambda row: sunshine_avg.get((row['temp_range_avg'], row['temp_range_min'], row['temp_range_max']), row['meteo_sunshine_%'])
    if pd.isna(row['meteo_sunshine_%']) else row['meteo_sunshine_%'], axis=1
)

df_cleaned['meteo_sunshine_duration'] = df_cleaned.apply(
    lambda row: duration_avg.get((row['temp_range_avg'], row['temp_range_min'], row['temp_range_max']), row['meteo_sunshine_duration'])
    if pd.isna(row['meteo_sunshine_duration']) else row['meteo_sunshine_duration'], axis=1
)

# Remplir les NaN pour meteo_pressure_avg
df_cleaned['meteo_pressure_avg'] = df_cleaned.apply(
    lambda row: pressure_avg.get(row['temp_range_avg'], row['meteo_pressure_avg'])
    if pd.isna(row['meteo_pressure_avg']) else row['meteo_pressure_avg'], axis=1
)

# Remplir les NaN pour meteo_evapotranspiration_Monteith
df_cleaned['meteo_evapotranspiration_Monteith'] = df_cleaned.apply(
    lambda row: evapotranspiration_avg.get(
        (row['temp_range_avg'], row['humidity_avg_range'], row['humidity_min_range']),
        row['meteo_evapotranspiration_Monteith']
    )
    if pd.isna(row['meteo_evapotranspiration_Monteith']) else row['meteo_evapotranspiration_Monteith'], axis=1
)


In [10]:
df_cleaned.isna().sum()

row_index                             0
meteo_rain_height                     0
meteo_temperature_min                 0
meteo_temperature_max                 0
meteo_temperature_avg                 0
meteo_pressure_avg                    0
meteo_humidity_min                    0
meteo_humidity_avg                    0
meteo_sunshine_duration              31
meteo_sunshine_%                     31
meteo_evapotranspiration_Monteith    43
meteo_evapotranspiration_grid         0
temp_range_avg                        0
temp_range_min                        0
temp_range_max                        0
humidity_avg_range                   29
humidity_min_range                   10
dtype: int64

In [11]:
df_cleaned.drop(columns=['temp_range_avg','temp_range_min','temp_range_max','humidity_avg_range','humidity_min_range'], inplace = True)
df_cleaned = df_cleaned.dropna()


In [12]:
df_cleaned.isna().sum()

row_index                            0
meteo_rain_height                    0
meteo_temperature_min                0
meteo_temperature_max                0
meteo_temperature_avg                0
meteo_pressure_avg                   0
meteo_humidity_min                   0
meteo_humidity_avg                   0
meteo_sunshine_duration              0
meteo_sunshine_%                     0
meteo_evapotranspiration_Monteith    0
meteo_evapotranspiration_grid        0
dtype: int64

In [13]:
df_cleaned.describe()

,row_index,meteo_rain_height,meteo_temperature_min,meteo_temperature_max,meteo_temperature_avg,meteo_pressure_avg,meteo_humidity_min,meteo_humidity_avg,meteo_sunshine_duration,meteo_sunshine_%,meteo_evapotranspiration_Monteith,meteo_evapotranspiration_grid
count,2.972050e+05,297205.000000,297205.000000,297205.000000,297205.000000,297205.000000,297205.000000,297205.000000,297205.000000,297205.000000,297205.000000,297205.000000
mean,2.968727e+06,1.768150,15.052554,27.464883,20.956624,1015.953122,41.934991,67.512047,518.259692,60.491107,4.830321,4.860810
std,4.881734e+05,5.204179,3.920289,5.012522,3.963402,2.391093,14.159878,13.433262,186.879606,21.213871,1.687088,1.804705
min,2.331795e+06,0.000000,-3.000000,4.400000,2.600000,991.000000,4.000000,24.000000,0.000000,0.000000,0.100000,0.200000
25%,2.492417e+06,0.000000,12.600000,24.000000,18.400000,1015.388870,32.000000,58.000000,425.094501,49.096428,3.749834,3.600000
50%,2.652417e+06,0.000000,15.200000,27.400000,20.900000,1015.990232,41.000000,69.000000,464.000000,55.000000,4.698833,4.700000
75%,3.451296e+06,0.600000,17.600000,30.900000,23.700000,1016.330934,52.000000,78.000000,674.297138,78.348705,6.200000,6.000000
max,3.610822e+06,281.700000,30.400000,43.300000,33.800000,1033.000000,99.000000,100.000000,919.000000,97.000000,9.000000,9.000000


In [14]:
df_cleaned.head()

,row_index,meteo_rain_height,meteo_temperature_min,meteo_temperature_max,meteo_temperature_avg,meteo_pressure_avg,meteo_humidity_min,meteo_humidity_avg,meteo_sunshine_duration,meteo_sunshine_%,meteo_evapotranspiration_Monteith,meteo_evapotranspiration_grid
0,2331795,7.4,12.8,23.2,17.1,1016.330934,57.0,85.0,462.070181,54.070896,3.126286,2.7
1,2331796,3.4,10.5,23.1,16.8,1015.800000,54.0,82.0,234.000000,26.000000,2.500000,2.5
3,2331798,12.8,5.8,17.1,11.9,1015.252176,59.0,83.0,142.000000,15.000000,1.700000,1.5
5,2331800,5.4,10.3,20.9,14.4,1015.252176,55.0,87.0,277.000000,30.000000,2.500000,2.5
6,2331801,3.4,10.5,23.1,16.8,1015.800000,54.0,82.0,234.000000,26.000000,2.500000,2.5
